# Settings

In [1]:
OUTPUT_CLEANED_FOLDER = "output/"

# Read data

In [2]:
import os 
import pandas as pd 

df_interaction = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_activity_interaction_deep_cleaned.csv"))
df_user = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_info_deep_cleaned.csv"))
df_activity = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "activity_info_deep_cleaned.csv"))

/tmp/ipykernel_2551492/1144630671.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_interaction = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_activity_interaction_deep_cleaned.csv"))


In [3]:
df_activity["id"]  = df_activity["id"].astype(str)

In [4]:
df_activity.head()

,id,title,type,desc,btc,createdTime,startTime,finishTime,deadlineTime,criteria
0,135,Tọa đàm đối thoại giữa sinh viên và lãnh đạo n...,Sinh hoạt chuyên đề,"Các bạn sinh viên thân mến, Gần đây, Trường ch...",Đại học Bách Khoa Hà Nội,2019-11-26 18:03:25,2019-11-26 18:00:00,2019-11-27 21:00:00,2019-11-27 21:00:00,"[{'CGroupId': 0, 'CId': 67, 'CName': 'Tham gia..."
1,136,Ngày hội tuyển dụng của công ty LG Electronics,Hội thảo hướng nghiệp,Công ty LG Electronics Việt Nam Hải Phòng (LGE...,Ban CTSV,2019-11-27 14:51:13,2019-11-29 08:30:00,2019-11-29 11:30:00,2019-11-29 11:30:00,"[{'CGroupId': 0, 'CId': 61, 'CName': 'Tham gia..."
2,137,Cổ vũ Giải bóng đá nam sinh viên các trường...,Hoạt động thể thao,Giải bóng đá sinh viên Hà Nội là giải đấu thườ...,Ban CTSV,2019-12-02 11:16:37,2019-12-02 08:00:00,2019-12-02 10:30:00,2019-12-02 10:30:00,"[{'CGroupId': 0, 'CId': 62, 'CName': 'Tham gia..."
3,146,"Lễ tổng kết cuộc thi ""Tuổi trẻ học tập và làm ...",Sinh hoạt chuyên đề,- Tiếp tục đẩy mạnh học tập và làm theo tư tưở...,Ban CTSV,2019-12-11 09:37:19,2019-12-14 18:00:00,2019-12-14 22:00:00,2019-12-14 22:00:00,"[{'CGroupId': 0, 'CId': 60, 'CName': 'Tham gia..."
4,147,Họp Chi bộ 12/2019,Hiến máu nhân đạo,Kiểm điểm đảng viên Triển khai công tác khác c...,Chi bộ SV II,2019-12-11 14:25:48,2019-12-08 17:00:00,2019-12-08 19:30:00,2019-12-08 19:30:00,"[{'CGroupId': 0, 'CId': 55, 'CName': 'Tham gia..."


In [5]:
df_interaction = df_interaction.astype(str)
df_interaction.dtypes

user_id            object
activity_id        object
accepted_status    object
dtype: object

In [6]:
df_interaction.head()

,user_id,activity_id,accepted_status
0,20186237,135,2
1,20170351,135,2
2,20191797,135,2
3,20174071,135,1
4,20193457,135,1


In [7]:
df_interaction["inter_score"] = 1

In [8]:
df_interaction

,user_id,activity_id,accepted_status,inter_score
0,20186237,135,2,1
1,20170351,135,2,1
2,20191797,135,2,1
3,20174071,135,1,1
4,20193457,135,1,1
...,...,...,...,...
3287967,20230352P,8969,2,1
3287968,20210381P,8969,2,1
3287969,20210126P,8969,2,1
3287970,20230327P,8969,2,1


In [9]:
df_all = df_interaction.merge(df_activity, left_on=["activity_id"], right_on=["id"])[["user_id", "activity_id", "inter_score", "createdTime"]]
df_all.head()

,user_id,activity_id,inter_score,createdTime
0,20186237,135,1,2019-11-26 18:03:25
1,20170351,135,1,2019-11-26 18:03:25
2,20191797,135,1,2019-11-26 18:03:25
3,20174071,135,1,2019-11-26 18:03:25
4,20193457,135,1,2019-11-26 18:03:25


In [10]:
df_all.tail()

,user_id,activity_id,inter_score,createdTime
3287967,20230352P,8969,1,2023-11-23 10:48:28
3287968,20210381P,8969,1,2023-11-23 10:48:28
3287969,20210126P,8969,1,2023-11-23 10:48:28
3287970,20230327P,8969,1,2023-11-23 10:48:28
3287971,20234387,8870,1,2023-10-17 12:12:00


In [11]:
df_all["createdTime"] = pd.to_datetime(df_all["createdTime"])

# Metrics 

In [12]:
import numpy as np
import math

def getHitRatio(ranklist, gtItem):
    if gtItem in ranklist:
        return 1
    return 0

def getNDCG(ranklist, gtItem):
    ar = np.array(ranklist)
    if gtItem in ar:
        return math.log(2) / math.log(np.where(ar == gtItem)[0][0] + 2)
    return 0


# Training Testing Split


In [13]:
def train_test_split_time(df , columns_1 , columns_2 , columns_time):
    '''
    we will use the columns_1 as the first level index
    sort the columns_2 as the target by columns_time
    '''
    
    retrieve_name = df[columns_1].value_counts()[df[columns_1].value_counts() > 5].keys()
    df_test = df[df[columns_1].isin(retrieve_name)]
    df_test = df_test.sort_values(columns_time).groupby(columns_1).tail(1)
    df_train = df.drop(index=df_test.index)
    
    if df_test.shape[0] + df_train.shape[0] == df.shape[0]:
        print('train_test_split succeed!! with df_train shape:(%d,%d), df_test shape:(%d,%d)'
              %(df_train.shape[0],df_train.shape[1],df_test.shape[0],df_test.shape[1])
             )
        return df_train, df_test, retrieve_name
    else:
        print('Oops, something wrong, with df_train shape : (%d,%d), df_test shape : (%d,%d)'
              %(df_train.shape[0],df_train.shape[1],df_test.shape[0],df_test.shape[1])
             )
        print('df_shape : (%d,%d)'
              %(df.shape[0],df.shape[1])
             )

In [14]:
df_train, df_test, retrieve = train_test_split_time(df_all, 'user_id','activity_id', 'createdTime')

train_test_split succeed!! with df_train shape:(3234660,4), df_test shape:(53312,4)


In [15]:
df_train[df_train["user_id"] == "20204876"].sort_values(by="createdTime")

,user_id,activity_id,inter_score,createdTime
449583,20204876,901,1,2020-09-18 11:32:04
1638608,20204876,907,1,2020-10-08 11:34:17
1286422,20204876,911,1,2020-10-14 17:41:22
2458868,20204876,921,1,2020-10-18 21:54:32
2838495,20204876,929,1,2020-10-22 16:43:32
...,...,...,...,...
3156415,20204876,8978,1,2023-11-25 21:58:12
2773923,20204876,8982,1,2023-11-27 08:40:36
2418474,20204876,9025,1,2023-12-08 15:44:03
2425039,20204876,9033,1,2023-12-11 16:23:32


In [16]:
df_test[df_test["user_id"] == "20204876"]

,user_id,activity_id,inter_score,createdTime
2803899,20204876,9075,1,2023-12-19 21:34:35


We will use the leave-one-out method to evaluate our testing data

Basically, we will generate N samples list that the test user hasn't read and add the ground truth into the list, and see how the recommendation system will score and rank the list

In [17]:
set(df_all["activity_id"].unique())

{'8783',
 '1366',
 '2006',
 '8265',
 '3092',
 '2976',
 '423',
 '440',
 '1142',
 '8804',
 '2072',
 '2275',
 '2104',
 '429',
 '3228',
 '1104',
 '8600',
 '5407',
 '8472',
 '1313',
 '3126',
 '8287',
 '8466',
 '2667',
 '1137',
 '6778',
 '3147',
 '1759',
 '2017',
 '3261',
 '6678',
 '8893',
 '6987',
 '8852',
 '2053',
 '584',
 '8515',
 '8451',
 '1741',
 '2417',
 '8335',
 '1478',
 '8531',
 '1485',
 '6591',
 '3027',
 '5495',
 '2465',
 '2440',
 '1187',
 '2442',
 '3179',
 '1028',
 '2611',
 '1652',
 '8351',
 '7212',
 '5455',
 '3150',
 '364',
 '355',
 '2863',
 '2068',
 '1284',
 '6842',
 '6702',
 '8297',
 '8708',
 '3171',
 '8468',
 '582',
 '1491',
 '7055',
 '8611',
 '2416',
 '1959',
 '8543',
 '7141',
 '581',
 '5509',
 '8606',
 '1167',
 '1886',
 '2896',
 '5462',
 '854',
 '2089',
 '5483',
 '5494',
 '1501',
 '7072',
 '8587',
 '7179',
 '1323',
 '9058',
 '2824',
 '1694',
 '6931',
 '2935',
 '1096',
 '7014',
 '2770',
 '3186',
 '5432',
 '7130',
 '2390',
 '8626',
 '8860',
 '8583',
 '294',
 '1713',
 '8467',
 '

In [18]:
len(set(df_all["activity_id"].unique()).difference(sorted(df_all.groupby("user_id")['activity_id'].apply(list)["20204876"])))

4310

In [19]:
import random

random.seed(73)

def get_sampling(df_target, df_source, numbers_of_N_sample, generate_negative = False):
    '''
    1.
    return: test_rating in the shape of [ [user_pid,article] * N ]
    2.
    from the target dataframe, sampling the negative sample from source dataframe
    return: list in the shape of [ [((user_id, ground truth)) , [numbers_of_N_sample(negative samples)]] * N]
    '''
    test_rating = df_target[['user_id','activity_id']].values.tolist()
    print('test_rating yield successfully!!!')
    
    if generate_negative:
        act_set = set(df_source["activity_id"].unique())
        mapping_user_act = df_source.groupby("user_id")['activity_id'].apply(list)
        negative_test_rating = []
        for i in range(df_target.shape[0]):
            drop_id = test_rating[i][0]
            list_ = random.sample(list(act_set.difference(mapping_user_act[drop_id])), numbers_of_N_sample)
            negative_test_rating.append([test_rating[i], list_])
        print('negative_test_rating yield successfully!!!')

        return test_rating, negative_test_rating
    else:
        return test_rating

In [20]:
print('''only for generating sample, if there is a sample file already, skip this cell in the future''')
# In the future just execute the next cell to get the test rating & negative sampling
# import time
# import pickle

# time1 = time.time()
# test_rating, negative_test_rating = get_sampling(df_test, df_train, 19, True)
# time2 = time.time()
# print('Took for %d seconds' %(time2-time1))
# with open("output/negative_test_rating_19N.pkl","wb") as f: #in write mode
#     pickle.dump(negative_test_rating, f)

only for generating sample, if there is a sample file already, skip this cell in the future


In [21]:
import time
import pickle

time1 = time.time()
test_rating = get_sampling(df_test, df_train, None)
with open("output/negative_test_rating_19N.pkl",'rb') as f: #in read mode, not in write mode, careful
    negative_test_rating = pickle.load(f)
    print('negative_test_rating yield successfully!!!')
    
time2 = time.time()
print('Took for %d seconds' %(time2-time1))

test_rating yield successfully!!!
negative_test_rating yield successfully!!!
Took for 0 seconds


# Create Features for Activity

In [22]:
df_activity_processed = df_activity.copy()
df_activity_processed.nunique()

id              4537
title           4532
type              19
desc            2948
btc              143
createdTime     4534
startTime       2312
finishTime      2391
deadlineTime    2383
criteria         381
dtype: int64

In [23]:
df_activity_processed.dtypes

id              object
title           object
type            object
desc            object
btc             object
createdTime     object
startTime       object
finishTime      object
deadlineTime    object
criteria        object
dtype: object

In [24]:
df_activity_processed.head()

,id,title,type,desc,btc,createdTime,startTime,finishTime,deadlineTime,criteria
0,135,Tọa đàm đối thoại giữa sinh viên và lãnh đạo n...,Sinh hoạt chuyên đề,"Các bạn sinh viên thân mến, Gần đây, Trường ch...",Đại học Bách Khoa Hà Nội,2019-11-26 18:03:25,2019-11-26 18:00:00,2019-11-27 21:00:00,2019-11-27 21:00:00,"[{'CGroupId': 0, 'CId': 67, 'CName': 'Tham gia..."
1,136,Ngày hội tuyển dụng của công ty LG Electronics,Hội thảo hướng nghiệp,Công ty LG Electronics Việt Nam Hải Phòng (LGE...,Ban CTSV,2019-11-27 14:51:13,2019-11-29 08:30:00,2019-11-29 11:30:00,2019-11-29 11:30:00,"[{'CGroupId': 0, 'CId': 61, 'CName': 'Tham gia..."
2,137,Cổ vũ Giải bóng đá nam sinh viên các trường...,Hoạt động thể thao,Giải bóng đá sinh viên Hà Nội là giải đấu thườ...,Ban CTSV,2019-12-02 11:16:37,2019-12-02 08:00:00,2019-12-02 10:30:00,2019-12-02 10:30:00,"[{'CGroupId': 0, 'CId': 62, 'CName': 'Tham gia..."
3,146,"Lễ tổng kết cuộc thi ""Tuổi trẻ học tập và làm ...",Sinh hoạt chuyên đề,- Tiếp tục đẩy mạnh học tập và làm theo tư tưở...,Ban CTSV,2019-12-11 09:37:19,2019-12-14 18:00:00,2019-12-14 22:00:00,2019-12-14 22:00:00,"[{'CGroupId': 0, 'CId': 60, 'CName': 'Tham gia..."
4,147,Họp Chi bộ 12/2019,Hiến máu nhân đạo,Kiểm điểm đảng viên Triển khai công tác khác c...,Chi bộ SV II,2019-12-11 14:25:48,2019-12-08 17:00:00,2019-12-08 19:30:00,2019-12-08 19:30:00,"[{'CGroupId': 0, 'CId': 55, 'CName': 'Tham gia..."


In [25]:
df_activity_processed["createdTime"]

0       2019-11-26 18:03:25
1       2019-11-27 14:51:13
2       2019-12-02 11:16:37
3       2019-12-11 09:37:19
4       2019-12-11 14:25:48
               ...         
4532    2023-12-20 13:49:42
4533    2023-12-20 16:01:48
4534    2023-12-20 21:49:56
4535    2023-12-20 22:32:40
4536    2023-12-21 14:32:13
Name: createdTime, Length: 4537, dtype: object

In [26]:
df_activity_processed["createdTimeYear"] = pd.to_datetime(df_activity_processed["createdTime"]).dt.year

In [27]:
df_activity_processed["createdTimeYear"].unique() # category features

array([2019, 2020, 2021, 2022, 2023], dtype=int32)

In [28]:
df_activity_processed["type"].unique() # category features

array(['Sinh hoạt chuyên đề', 'Hội thảo hướng nghiệp',
       'Hoạt động thể thao', 'Hiến máu nhân đạo',
       'Hỗ trợ tổ chức sự kiện', 'Hoạt động văn nghệ',
       'Thực tập tại doanh nghiệp', 'Tham gia nghiên cứu LAB',
       'Sinh hoạt công dân', 'Hội thảo khoa học', 'Khóa học kỹ năng mềm',
       'Họp lớp', 'Cuộc thi khoa học công nghệ', 'Tình nguyện tại chỗ',
       'Hoạt động khảo sát', 'Khai báo online', 'Sinh hoạt câu lạc bộ',
       'Ủng hộ, quyên góp', 'Giáo dục Chính trị & Tư tưởng'], dtype=object)

In [29]:
df_activity_processed["btc"].unique() # too much -> concat to title, desc

array(['Đại học Bách Khoa Hà Nội', 'Ban CTSV', 'Chi bộ SV II',
       'Tổ hỗ trợ, tư vấn học tập và tâm lý', 'Tổ Văn Thể Mỹ',
       'Viện Ngoại Ngữ',
       'Liên chi Hội Sinh viên Viện Công nghệ Thông tin và Truyền thông',
       'Hội Sinh Viên', 'Viện Điện Tử - Viễn Thông',
       'Đoàn TNCS Hồ Chí Minh ĐH BKHN', 'BCH LCĐ - LCH SV Viện Điện',
       'Ban TCKT ĐTN & Ban TCXDH HSV', 'Ban Phát triển Thư viện',
       'Liên chi Đoàn - Liên chi Hội sinh viên Chương trình Đào tạo Quốc tế',
       'Viện Toán ứng dụng và Tin học',
       'Liên chi đoàn Viện Điện tử - Viễn thông',
       'Liên chi hội Sinh viên Viện Vật lý Kỹ thuật', 'SIE',
       'Liên chi Đoàn - Liên chi Hội Sinh viên - Viện Ngoại Ngữ',
       'Ban Truyền thông - VNN', 'Liên chi đoàn Viện CNTT&TT',
       'Viện Kinh tế & Quản lý', 'Ban Tuyển sinh',
       'Viện Sư phạm Kỹ thuật', 'Viện Điện',
       'Trường Công nghệ Thông tin và Truyền thông',
       'Viện KH&KT Vật liệu',
       'Liên Chi Đoàn-Liên Chi Hội Sinh viên Viện

In [30]:
df_activity_processed["createdTime"].min(), df_activity_processed["createdTime"].max() # categorize based on years

('2019-11-26 18:03:25', '2023-12-21 14:32:13')

In [31]:
df_activity_processed["criteria"][0]

"[{'CGroupId': 0, 'CId': 67, 'CName': 'Tham gia các diễn đàn sinh viên, các buổi toạ đàm với lãnh đạo Viện, Trường', 'CType': 0, 'CMaxPoint': 3.0, 'CStatus': 0}]"

In [32]:
import re

def clean_json(string):
    p = re.compile('(?<!\\\\)\'')
    return p.sub('\"', string)

In [33]:
import json
citeria_list = {}

for index, row in df_activity_processed.iterrows():
    criteria = row['criteria']
    try:
        json_parsed = json.loads(clean_json(criteria))
        for crite in json_parsed:
                citeria_list[crite["CId"]] = {"id": crite["CId"],
                                            "name": crite["CName"],
                                            "point": crite["CMaxPoint"]}
    except Exception as e:
         pass 
         # Just ignore them, We cannot parse them :)

In [34]:
len(citeria_list) # Categorical 

49

In [35]:
## Bi-Encoder for encoding text
act_id_to_text = {}

for index, row in df_activity_processed.iterrows():
    title = str(row['title']).strip() if row['title'] is not None else ""
    desc = str(row["desc"]).strip() if row['desc'] is not None else ""
    btc = str(row["btc"]).strip() if row['btc'] is not None else ""
    
    act_id_to_text[row["id"]] = title + "\n\n" + desc + "\n\n" + btc

In [36]:
print("You do not need run this if you already have the embeddings")
# # INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!

# Necessary libraries

# from tqdm import tqdm 
# from sentence_transformers import SentenceTransformer
# from pyvi import ViTokenizer

# SENTENCE_PER_BATCH = 64
# model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
# len_all = len(act_id_to_text)

# list_key = []
# list_embedding = []
# count = 0
# batch_processed = []

# for key, value in tqdm(act_id_to_text.items()):
#     list_key.append(key)
#     batch_processed.append(ViTokenizer.tokenize(value))
#     count += 1
#     if not count % SENTENCE_PER_BATCH or count == len_all:
#         embeddings = model.encode(batch_processed)
#         list_embedding.extend(embeddings)
#         batch_processed = []

# act_id_to_emb = {}
# for key, emb in zip(list_key, list_embedding):
#     act_id_to_emb[key] = emb

# import pickle
# with open("output/activity_embedding.pkl", "wb") as fOut:
#     pickle.dump(act_id_to_emb, fOut)

You do not need run this if you already have the embeddings


In [37]:
import pickle
with open("output/activity_embedding.pkl", "rb") as fOut:
    act_id_to_emb = pickle.load(fOut)

In [38]:
# Create one-hot vector for type
type_map_one_hot = {}

nunique = df_activity_processed["type"].nunique()
for idx, data in enumerate(df_activity_processed["type"].unique()):
    type_map_one_hot[data] = idx
    vector = [0] * nunique
    vector[idx] = 1.0
    type_map_one_hot[data] = vector

In [39]:
# For startYear
start_year_map_one_hot = {}

nunique = df_activity_processed["createdTimeYear"].nunique()
for idx, data in enumerate(df_activity_processed["createdTimeYear"].unique()):
    start_year_map_one_hot[data] = idx
    vector = [0] * nunique
    vector[idx] = 1.0
    start_year_map_one_hot[data] = vector

In [40]:
# criteria
criteria_map_id = {}
for idx, data in enumerate(list(citeria_list.keys())):
    criteria_map_id[data] = idx

In [41]:
df_activity_processed["criteria"].isnull().sum()

0

In [42]:
def criteria_map_one_hot(string):
    results = [0] * (len(criteria_map_id) + 1)
    try:
        json_parsed = json.loads(clean_json(string))
        for crite in json_parsed:
            results[criteria_map_id[crite["CId"]]] = 1.0
    except Exception as e:
        results[-1] = 1.0 

    return results

In [43]:
# final features 
act_id_to_features = {} # final features
for _, row in df_activity_processed.iterrows():
    features = []
    activity_id = row["id"]
    # text
    features.extend(act_id_to_emb[activity_id])
    # type one hot 
    features.extend(type_map_one_hot[row["type"]])
    # criteria features
    features.extend(criteria_map_one_hot(row["criteria"]))
    # start year
    features.extend(start_year_map_one_hot[row["createdTimeYear"]])

    act_id_to_features[activity_id] = features

In [44]:
len(act_id_to_features["8300"])

842

# Create Features for Users

In [45]:
df_user_processed = df_user.copy()
df_user_processed.nunique()

id          59273
fullName    37730
email       59273
faculty        18
dtype: int64

In [46]:
df_user_processed["faculty"].unique()

array(['Trường Vật liệu', 'Viện Kinh tế và Quản lý',
       'Trường Điện - Điện tử', 'Trường Hóa và Khoa học Sự sống',
       'Viện Toán ứng dụng và Tin học', 'Trường Cơ khí', 'Viện Ngoại ngữ',
       'Trường Công nghệ Thông tin và Truyền thông',
       'Viện Vật lý Kỹ thuật', 'Viện Sư phạm Kỹ thuật', 'SIE', 'Ban CTSV',
       nan, 'Trường Công nghệ thông tin & Truyền thông',
       'Viện Vật lý kỹ thuật', 'Trường Vật Liệu',
       'Trung tâm Đào tạo liên tục', 'Trường Hóa và Khoa học sự sống',
       'Trường Cơ Khí'], dtype=object)

In [47]:
df_user_processed["faculty"].value_counts(dropna=False)

faculty
Trường Cơ khí                                 13987
Trường Điện - Điện tử                         13482
Trường Hóa và Khoa học Sự sống                 8841
Trường Công nghệ Thông tin và Truyền thông     7565
Viện Kinh tế và Quản lý                        3944
Trường Vật liệu                                2867
Viện Toán ứng dụng và Tin học                  1844
Viện Ngoại ngữ                                 1646
Trường Công nghệ thông tin & Truyền thông      1175
Viện Vật lý Kỹ thuật                           1126
SIE                                             914
Trường Vật Liệu                                 775
Viện Sư phạm Kỹ thuật                           424
Trung tâm Đào tạo liên tục                      315
Viện Vật lý kỹ thuật                            298
Ban CTSV                                         47
NaN                                              11
Trường Hóa và Khoa học sự sống                    8
Trường Cơ Khí                                     4
Name

In [48]:
mapping_similar = {"Trường Cơ khí": "Trường Cơ Khí",
                   "Trường Hóa và Khoa học sự sống": "Trường Hóa và Khoa học Sự sống",
                   "Trường Công nghệ thông tin & Truyền thông": "Trường Công nghệ Thông tin và Truyền thông",
                   "Viện Vật lý kỹ thuật": "Viện Vật lý Kỹ thuật"}


df_user_processed["faculty"] = df_user_processed["faculty"].apply(lambda x :  mapping_similar[x] if x in mapping_similar else x)
df_user_processed["faculty"].value_counts(dropna=False)

faculty
Trường Cơ Khí                                 13991
Trường Điện - Điện tử                         13482
Trường Hóa và Khoa học Sự sống                 8849
Trường Công nghệ Thông tin và Truyền thông     8740
Viện Kinh tế và Quản lý                        3944
Trường Vật liệu                                2867
Viện Toán ứng dụng và Tin học                  1844
Viện Ngoại ngữ                                 1646
Viện Vật lý Kỹ thuật                           1424
SIE                                             914
Trường Vật Liệu                                 775
Viện Sư phạm Kỹ thuật                           424
Trung tâm Đào tạo liên tục                      315
Ban CTSV                                         47
NaN                                              11
Name: count, dtype: int64

In [49]:
# For startYear
faculty_map_one_hot = {}

nunique = df_user_processed["faculty"].nunique(dropna=False)
for idx, data in enumerate(df_user_processed["faculty"].unique()):
    faculty_map_one_hot[data] = idx
    vector = [0] * nunique
    vector[idx] = 1.0
    faculty_map_one_hot[data] = vector

In [50]:
faculty_map_one_hot[np.nan]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0]

In [51]:
df_user_processed

,id,fullName,email,faculty
0,20186237,Bùi Ánh Nguyệt,nguyet.ba186237@sis.hust.edu.vn,Trường Vật liệu
1,20170351,Bùi Bích Hoà,hoa.bb170351@sis.hust.edu.vn,Viện Kinh tế và Quản lý
2,20191797,Bùi Công Duy,duy.bc191797@sis.hust.edu.vn,Trường Điện - Điện tử
3,20174071,Bùi Công Minh,minh.bc174071@sis.hust.edu.vn,Trường Điện - Điện tử
4,20193457,Bùi Hà Trang,trang.bh193457@sis.hust.edu.vn,Trường Hóa và Khoa học Sự sống
...,...,...,...,...
59268,20230330P,Nguyễn Đình Lộc,loc.nd230330p@sis.hust.edu.vn,Trung tâm Đào tạo liên tục
59269,20230301P,Nguyễn Thiên Anh,anh.nt230301p@sis.hust.edu.vn,Trung tâm Đào tạo liên tục
59270,20230538P,Trịnh Thu Ngân,ngan.tt230538p@sis.hust.edu.vn,Trung tâm Đào tạo liên tục
59271,20230358P,Đặng Quang Thành,thanh.dq230358p@sis.hust.edu.vn,Trung tâm Đào tạo liên tục


In [52]:
import re
temp_id_df = df_user_processed["id"].apply(lambda x: re.findall(r'\d+', x)[0]).astype(int)

In [53]:
temp_id_df.count(), temp_id_df.drop_duplicates().count() #P vs not P

(59273, 59112)

In [54]:
def school_year_calculate(temp_id):
    bin_list = [20120000, 20130000, 20140000, 20150000,
                20160000, 20170000, 20180000, 20190000,
                20200000, 20210000, 20220000, 20230000,
                20240000]
    
    corresponding_school_year = ["K57", "K58", "K59", "K60",
                                 "K61", "K62", "K63", "K64",
                                  "K65", "K66", "K67", "K68" ]
    
    for i in range(len(bin_list) - 1):
        if temp_id >= bin_list[i] and temp_id < bin_list[i+1]:
            return corresponding_school_year[i] 

In [55]:
df_user_processed["schoolYear"] = temp_id_df.apply(school_year_calculate)

In [56]:
df_user_processed["schoolYear"].value_counts(dropna=False)

schoolYear
K68    9026
K66    7803
K67    7418
K65    7378
K64    7127
K63    6049
K62    5843
K61    4534
K60    3600
K59     482
K58      12
K57       1
Name: count, dtype: int64

In [57]:
# For schoolYear
school_year_map_one_hot = {}

nunique = df_user_processed["schoolYear"].nunique(dropna=False)
for idx, data in enumerate(df_user_processed["schoolYear"].unique()):
    school_year_map_one_hot[data] = idx
    vector = [0] * nunique
    vector[idx] = 1.0
    school_year_map_one_hot[data] = vector

In [58]:
# final features 
user_id_to_features = {} # final features
for _, row in df_user_processed.iterrows():
    features = []
    user_id = row["id"]
    # faculty one hot 
    features.extend(faculty_map_one_hot[row["faculty"]])
    # start year
    features.extend(school_year_map_one_hot[row["schoolYear"]])

    user_id_to_features[user_id] = features

In [59]:
len(user_id_to_features["20204876"])

27

# Hybrid Recommendation System


In [60]:
user_id_map = {i:v for i, v in enumerate(df_all["user_id"].unique())}
inverse_user_id_map = {v:i for i, v in enumerate(df_all["user_id"].unique())}
activity_id_map = {i:v for i, v in enumerate(df_all["activity_id"].unique())}
inverse_activity_id_map = {v:i for i, v in enumerate(df_all["activity_id"].unique())}

In [61]:
import random 
random.seed(73)
np.random.seed(2023)

df_train_map = df_train.copy()
df_test_map = df_test.copy()
df_all_map = df_all.copy()


df_train_map['user_id'] = df_train_map['user_id'].map(inverse_user_id_map)
df_train_map['activity_id'] = df_train_map['activity_id'].map(inverse_activity_id_map)
df_test_map['user_id'] = df_test_map['user_id'].map(inverse_user_id_map)
df_test_map['activity_id'] = df_test_map['activity_id'].map(inverse_activity_id_map)
df_all_map['user_id'] = df_all['user_id'].map(inverse_user_id_map)
df_all_map['activity_id'] = df_all['activity_id'].map(inverse_activity_id_map)


### creating a mapping list for testing data   
test_rating_map = []
for i in range(len(test_rating)):
    test_rating_map.append([inverse_user_id_map[test_rating[i][0]], inverse_activity_id_map[test_rating[i][1]]])


### creating a mapping list for negative testing data
negative_test_rating_map = []
for i in range(len(negative_test_rating)):
    negative_test_rating_map.append([
                                     [inverse_user_id_map[negative_test_rating[i][0][0]],
                                      inverse_activity_id_map[negative_test_rating[i][0][1]]],
                                     list(map(lambda w: inverse_activity_id_map[w], negative_test_rating[i][1]))])

### append negative sample into df_training_map

NEGATIVE_FACTOR = 10 # 50 

negative_training_size = NEGATIVE_FACTOR * df_train_map.shape[0]

ng_user_id = np.random.randint(low = 0, 
                               high = max(user_id_map.keys()), 
                               size = negative_training_size)

ng_act_id = np.random.randint(low = 0, 
                              high = max(activity_id_map.keys()), 
                              size = negative_training_size) # including false negative ??

act_set = set(df_all["activity_id"].unique())
mapping_user_act = df_all.groupby("user_id")['activity_id'].apply(list)

df_ngt = pd.DataFrame(columns=['user_id','activity_id', 'inter_score'],
                      data=np.array([ng_user_id, ng_act_id, [0] * negative_training_size]).T)

df_train_map = df_train_map.drop_duplicates(subset=['user_id','activity_id'], keep = 'first')
df_train_map = pd.concat([df_train_map[['user_id','activity_id', 'inter_score']], df_ngt] , ignore_index=True).reset_index()[["user_id", 'activity_id', 'inter_score']]

In [62]:
df_train_map = df_train_map.sort_values(by="inter_score", ascending=False)
df_train_map

,user_id,activity_id,inter_score
0,0,0,1
2155789,42684,1082,1
2155791,45820,1082,1
2155792,42693,1082,1
2155793,42696,1082,1
...,...,...,...
14015886,27821,2069,0
14015885,8672,2178,0
14015884,15804,2629,0
14015883,23368,1673,0


In [63]:
df_train_map = df_train_map.drop_duplicates(["user_id", "activity_id"], keep= "first") # Not quite right, negative samples could be dedup but maybe it alright
df_train_map

,user_id,activity_id,inter_score
0,0,0,1
2155789,42684,1082,1
2155791,45820,1082,1
2155792,42693,1082,1
2155793,42696,1082,1
...,...,...,...
14015886,27821,2069,0
14015885,8672,2178,0
14015884,15804,2629,0
14015883,23368,1673,0


# Modeling

In [64]:
# https://github.com/TeddyWang0202/recommendation_system/blob/master/Hybrid.ipynb

In [65]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[7], 'GPU')

2023-12-24 11:12:20.805778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 11:12:20.805853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 11:12:20.807068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-24 11:12:20.813778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-24 11:12:22.071696: W tensorflow/compiler/tf2

In [66]:
import keras
keras.__version__

'2.15.0'

In [67]:
from keras.layers import Embedding


num_items = len(act_id_to_emb.items())
### Try create embedding matrix
activity_vector_create = act_id_to_emb.copy()
activity_vector_create[-1] = [0.0] * 768

activity_map_vector = {(inverse_activity_id_map[k] if k in inverse_activity_id_map else -1): v  for k, v in activity_vector_create.items()}
activity_map_vector.pop(-1)

print(len(activity_map_vector))
input_len = 768
embedding_matrix = np.zeros((num_items, input_len), dtype=np.float32)
for i in activity_map_vector.keys():
    embedding_matrix[i] = np.array(activity_map_vector[i][:input_len])

bert_embedding_layer = Embedding(num_items,
                            input_len,
                            weights=[embedding_matrix],
                            name = 'BERT_embedding_vector',
                            trainable=False)


4505


In [68]:
from keras import initializers
from keras.models import Model
from keras.regularizers import l2
from keras.layers import multiply, Input, Reshape, Flatten, Dense, Concatenate


def init_normal(shape, dtype=None):
    return initializers.VarianceScaling(scale=0.01, 
    mode='fan_in', distribution='normal')(shape)

def get_model(num_users, num_items, latent_dim, embedding_layer, regs=[0, 0]):
    
    ''' Deep part : user-item-log embedding part '''
    ### define placeholder.
    user_id_input = Input(shape=[1], name='user')
    item_id_input = Input(shape=[1], name='item')

    ### define embedding size and layers.

    user_embedding = Embedding(output_dim = latent_dim, 
                               input_dim = num_users,
                               input_length= 1, 
                               name='user_embedding',
                               embeddings_regularizer = l2(regs[0]),
                               embeddings_initializer = init_normal)(user_id_input)
    
    item_embedding = Embedding(output_dim = latent_dim, 
                               input_dim = num_items,
                               input_length=1, 
                               name='item_embedding',
                               embeddings_regularizer = l2(regs[1]),
                               embeddings_initializer = init_normal)(item_id_input)

    user_vecs = Reshape([latent_dim])(user_embedding)
    item_vecs = Reshape([latent_dim])(item_embedding)
    
    mf = multiply([user_vecs, item_vecs])

    ''' Wide part : item embedding vector part '''
    ### input extracted matrix(here use embedding) and concatenate with one single DNN layer 
    item_vector = Flatten()(embedding_layer(item_id_input))
    item_vector_dense = Dense(768, 
                              activation='relu')(item_vector)
    
    '''Concatenate the deep and wide part'''
    
    concate_vector = Concatenate()([mf,
                                    item_vector_dense,
                                   ])
    
    ### The prediction, which we calculate the loss function with ground truth and optimize.
    y_hat= Dense(1, 
                 activation='selu', 
                 kernel_initializer='lecun_uniform', 
                 name = 'prediction')(concate_vector)
    
    
    model = Model(inputs=[user_id_input, item_id_input], 
                  outputs=y_hat)
    
    return model


In [69]:
from keras.optimizers import Adam
import heapq

num_users = len(user_id_map.items())
num_items = len(activity_id_map.items())

topK = 5
verbose = 0
latent_dim = 15
epochs = 100
batch_size = 4096
evaluation_threads = 1
best_hr, best_ndcg, best_iter = -1, -1, -1
model_out_file = 'models/DWN_%d_%d.h5' %(latent_dim, time.time())
testRatings, testNegatives = test_rating_map, negative_test_rating_map
learning_rate = 0.001
model = get_model(num_users, 
                  num_items, 
                  latent_dim, 
                  bert_embedding_layer, 
                  regs=[0,0])


2023-12-24 11:12:24.385062: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37954 MB memory:  -> device: 7, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [70]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 item (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 15)                889080    ['user[0][0]']                
                                                                                                  
 item_embedding (Embedding)  (None, 1, 15)                67575     ['item[0][0]']                
                                                                                              

In [71]:
from tqdm import tqdm

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='bce')
hr_list = []
loss_list = []
patience = 5
early_stop =True


# Generate training instances
user_input, item_input = df_train_map["user_id"], df_train_map["activity_id"]
labels = df_train_map["inter_score"]

print('start training...')
for epoch in range(epochs):
    t1 = time.time()
    print('training for the %d epoch' % (epoch))    
    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                         np.array(labels), # labels 
                         batch_size = batch_size,
                         validation_data = ([df_test_map["user_id"],df_test_map["activity_id"]],
                                            df_test_map["inter_score"]) ,
                         epochs = 1, verbose = verbose, shuffle = True)
    t2 = time.time()

    # Evaluation
    hits, ndcgs = [], []
    for idx in tqdm(range(len(testRatings))):
        rating = testRatings[idx]
        ### rating should be like (  [user_id,article_pid(only one article, which is ground truth)]  ) 
        items = testNegatives[idx][1]
        ### items should be like (  [article_pid(199 ones, which are ones the reader haven't read)]  )
        u = rating[0]
        gtItem = rating[1]
        items.append(gtItem)
        map_item_score = {}
        users = np.full(len(items), u, dtype = 'int32')

        predictions = model.predict([users,
                                     np.array(items)], 
                                     batch_size=20, verbose=0)
    
        for i in range(len(items)):
            item = items[i]
            map_item_score[item] = predictions[i]
        items.pop()
        ranklist = heapq.nlargest(topK, map_item_score, key=map_item_score.get)

        hr = getHitRatio(ranklist, gtItem)
        ndcg = getNDCG(ranklist, gtItem)

        hits.append(hr)
        ndcgs.append(ndcg)

    hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['val_loss'][0]    
    hr_list.append(hr)
    loss_list.append(loss)
    # Using patience to set the early stopping.
    # Always to save the model with minimun loss.
    if hr < np.max(hr_list):
        patience_count += 1
    else:
        patience_count = 0
        best_hr, best_ndcg, best_iter = hr, ndcg, epoch
        model.save_weights(model_out_file, overwrite=True)
    if (early_stop) and (patience_count == patience):
        break
    if epoch % 1 == 0:
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, val_loss = %.4f [%.1f s]' 
            % (epoch,  t2-t1, hr, ndcg, loss, time.time()-t2))

print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))

start training...
training for the 0 epoch


2023-12-24 11:12:28.589148: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-24 11:12:30.402281: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f97e989e5e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-24 11:12:30.402357: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2023-12-24 11:12:30.408023: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-24 11:12:30.480164: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1703391150.593516 2552506 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
 54%|█████▍    | 28888/53312 [49:49<40:49,  9.97i